In [1]:
# Shell Plus Model Imports
from callingcards.callingcards.models.Background import Background
from callingcards.callingcards.models.CCExperiment import CCExperiment
from callingcards.callingcards.models.CCTF import CCTF
from callingcards.callingcards.models.ChrMap import ChrMap
from callingcards.callingcards.models.Gene import Gene
from callingcards.callingcards.models.HarbisonChIP import HarbisonChIP
from callingcards.callingcards.models.Hops import Hops
from callingcards.callingcards.models.HopsReplicateSig import HopsReplicateSig
from callingcards.callingcards.models.KemmerenTFKO import KemmerenTFKO
from callingcards.callingcards.models.McIsaacZEV import McIsaacZEV
from callingcards.callingcards.models.PromoterRegions import PromoterRegions
from callingcards.callingcards.models.QcManualReview import QcManualReview
from callingcards.callingcards.models.QcMetrics import QcMetrics
from callingcards.callingcards.models.QcR1ToR2Tf import QcR1ToR2Tf
from callingcards.callingcards.models.QcR2ToR1Tf import QcR2ToR1Tf
from callingcards.callingcards.models.QcTfToTransposon import QcTfToTransposon
from callingcards.users.models import User
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission
from django.contrib.contenttypes.models import ContentType
from django.contrib.sessions.models import Session
from rest_framework.authtoken.models import Token, TokenProxy
# Shell Plus Django Imports
from django.db import models
from django.core.cache import cache
from django.conf import settings
from django.contrib.auth import get_user_model
from django.db import transaction
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery
from django.db import models
# setup django to work in the notebook
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
def get_promoter_ids():
    result = HarbisonChIP.objects.all().select_related(
                'tf',
                'gene',
                'gene__genepromoter'
            )\
            .annotate(
                tf_locus_tag=models.F('tf__locus_tag'),
                tf_gene=models.F('tf__gene'),
                target_locus_tag=models.F('gene__locus_tag'),
                target_gene=models.F('gene__gene'),
                target_gene_id=models.F('gene_id'),
                binding_signal=models.F('pval'),
                promoter_id=models.F('gene__genepromoter__id'),
                experiment=models.Value('harbison'))\
            .values('tf_id', 'tf_locus_tag', 'tf_gene',
                    'target_gene_id', 'target_locus_tag', 'target_gene', 
                    'binding_signal', 'experiment', 'promoter_id')[:5]
    
    return result

results = get_promoter_ids()
    

In [3]:
for record in results:
    print(record)

{'tf_id': 962, 'tf_locus_tag': 'YDL106C', 'tf_gene': 'PHO2', 'target_locus_tag': 'YAL001C', 'target_gene': 'TFC3', 'target_gene_id': 87, 'binding_signal': Decimal('0.6210000000000000000000000000'), 'promoter_id': 110, 'experiment': 'harbison'}
{'tf_id': 962, 'tf_locus_tag': 'YDL106C', 'tf_gene': 'PHO2', 'target_locus_tag': 'YAL001C', 'target_gene': 'TFC3', 'target_gene_id': 87, 'binding_signal': Decimal('0.6210000000000000000000000000'), 'promoter_id': 111, 'experiment': 'harbison'}
{'tf_id': 962, 'tf_locus_tag': 'YDL106C', 'tf_gene': 'PHO2', 'target_locus_tag': 'YAL001C', 'target_gene': 'TFC3', 'target_gene_id': 87, 'binding_signal': Decimal('0.6210000000000000000000000000'), 'promoter_id': 9574, 'experiment': 'harbison'}
{'tf_id': 962, 'tf_locus_tag': 'YDL106C', 'tf_gene': 'PHO2', 'target_locus_tag': 'YAL002W', 'target_gene': 'VPS8', 'target_gene_id': 86, 'binding_signal': Decimal('0.1022000000000000000000000000'), 'promoter_id': 108, 'experiment': 'harbison'}
{'tf_id': 962, 'tf_locu

In [2]:
from callingcards.callingcards.filters import PromoterRegionsFilter
import pandas as pd

def callingcards_with_metrics(query_params_dict):

    filtered_promoters = PromoterRegionsFilter(
        query_params_dict,
        queryset=PromoterRegions.objects.all())


    promoter_res = filtered_promoters.qs\
        .annotate(promoter_id=F('id'),
                  promoter_source=F('source'))
    promoter_df = pd.DataFrame.from_records(promoter_res.values())
    promoter_df = promoter_df[['promoter_id', 'promoter_source']]
    
    experiment_res = filtered_promoters.qs\
        .calling_cards_experiment(**query_params_dict)
    experiment_df = pd.DataFrame.from_records(experiment_res)
    experiment_df = experiment_df[['promoter_id', 'promoter_source',
                                   'experiment_hops', 'experiment_id']]
    
    return experiment_df


In [3]:
query_params_dict = {'promoter_source':'yiming', 'experiment_id':'75'}

filtered_promoters = PromoterRegionsFilter(
        query_params_dict,
        queryset=PromoterRegions.objects.all())


#callingcards_with_metrics(query_params_dict)

In [4]:
experiment_res = filtered_promoters.qs\
        .calling_cards_experiment(background_source='adh1', 
                                  experiment_id='75')
# experiment_df = pd.DataFrame.from_records(experiment_res)
# experiment_df = experiment_df[['promoter_id', 'promoter_source',
#                                    'experiment_hops', 'experiment_id']]

In [5]:
print(experiment_res.query)

SELECT "promoter_regions"."id" AS "promoter_id", COUNT(T5."id") AS "experiment_hops", T5."experiment_id" AS "experiment_id", "promoter_regions"."source" AS "promoter_source" FROM "promoter_regions" INNER JOIN "chr_map" ON ("promoter_regions"."chr_id" = "chr_map"."id") INNER JOIN "hops" ON ("chr_map"."id" = "hops"."chr_id") INNER JOIN "hops" T5 ON ("chr_map"."id" = T5."chr_id") WHERE ("promoter_regions"."source" = yiming AND "hops"."chr_id" = ("promoter_regions"."chr_id") AND "hops"."start" >= ("promoter_regions"."start") AND "hops"."start" <= ("promoter_regions"."end") AND T5."experiment_id" = 75) GROUP BY "promoter_regions"."id", T5."experiment_id", "promoter_regions"."source"


In [15]:
from django.db.models import Count, F, Q

In [16]:
print(experiment_hops.query)

SELECT "promoter_regions"."id" AS "promoter_id", "promoter_regions"."source" AS "promoter_source" FROM "promoter_regions"


In [ ]:
experiment_df = pd.DataFrame.from_records(experiment_res.values())

In [2]:
from callingcards.callingcards.utils.callingcards_with_metrics import callingcards_with_metrics

In [3]:
res_df = callingcards_with_metrics({'promoter_source':'yiming', 'background_source':'adh1', 'experiment_id':'75'})

In [21]:
from django.db.models import OuterRef, Subquery, Count, F, Q
import time

# Replace this with your pre-filtered Hops queryset
hops_queryset = Hops.objects.filter(experiment_id=75)

# Replace this with your desired PromoterRegions queryset
promoter_regions_queryset = PromoterRegions.objects.filter(source='yiming')

consider_strand = False  # Set this to True if you want to consider the strand

results = []
start_time = time.time()
for promoter_region in promoter_regions_queryset:
    # Create a base queryset for the Hops that match the chr, start, and end of the current promoter region
    matching_hops = hops_queryset.filter(
        chr_id=promoter_region.chr_id,
        start__gte=promoter_region.start,
        start__lte=promoter_region.end,
    )

    # If consider_strand is True, filter the matching Hops further by the strand
    if consider_strand:
        matching_hops = matching_hops.filter(strand=promoter_region.strand)

    # Count the number of matching Hops
    hops_count = matching_hops.count()

    # Append the result to the results list
    results.append(
        {
            'promoter_id': promoter_region.id,
            'experiment_hops': hops_count,
            'experiment_id': 75,  # Replace this with the actual experiment ID if it's not always 75
            'promoter_source': promoter_region.source,
        }
    )

print(time.time()-start_time)


9.649736404418945


In [7]:
res_df

,promoter_id,experiment_id,background_source,promoter_source,background_hops,background_total_hops,experiment_hops,experiment_total_hops,callingcards_enrichment,poisson_pval,hypergeometric_pval
0,9491,75,adh1,yiming,2,103922,0,8605,0.0,0.306223,0.147094
1,9492,75,adh1,yiming,4,103922,0,8605,0.0,0.412106,0.272553
2,9493,75,adh1,yiming,1,103922,0,8605,0.0,0.246331,0.076471
3,9494,75,adh1,yiming,1,103922,0,8605,0.0,0.246331,0.076471
4,9495,75,adh1,yiming,11,103922,0,8605,0.0,0.670713,0.583188
...,...,...,...,...,...,...,...,...,...,...,...
6704,16195,75,adh1,yiming,0,103922,0,8605,0.0,0.181269,1.000000
6705,16196,75,adh1,yiming,0,103922,0,8605,0.0,0.181269,1.000000
6706,16197,75,adh1,yiming,0,103922,0,8605,0.0,0.181269,1.000000
6707,16198,75,adh1,yiming,0,103922,0,8605,0.0,0.181269,1.000000


In [13]:
from callingcards.callingcards.filters import PromoterRegionsFilter,BackgroundFilter,HopsFilter

(PromoterRegionsFilter.Meta.fields +
                          BackgroundFilter.Meta.fields +
                          HopsFilter.Meta.fields +
                          ['consider_background'])

['chr_ucsc',
 'start',
 'end',
 'strand',
 'target_locus_tag',
 'target_gene',
 'score',
 'promoter_source',
 'id',
 'uploader',
 'uploadDate',
 'modified',
 'modifiedBy',
 'chr',
 'start',
 'end',
 'strand',
 'depth',
 'background_source',
 'tf_id',
 'tf_locus_tag',
 'tf_gene',
 'experiment',
 'experiment_id',
 'consider_background']

In [35]:
Hops.objects.all().experiment.unique()

AttributeError: 'QuerySet' object has no attribute 'experiment'

In [33]:
print(filtered_experiment.qs.query)

SELECT "hops"."id", "hops"."uploader_id", "hops"."uploadDate", "hops"."modified", "hops"."modifiedBy_id", "hops"."chr_id", "hops"."start", "hops"."end", "hops"."strand", "hops"."depth", "hops"."experiment_id" FROM "hops" WHERE "hops"."experiment_id" = 75


In [36]:
unique_experiment_ids = Hops.objects.all().values_list('experiment_id', flat=True).distinct()

In [37]:
leunique_experiment_ids

<QuerySet [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, '...(remaining elements truncated)...']>

In [38]:
len(unique_experiment_ids)

100

In [39]:
hops_queryset = Hops.objects.all()

unique_experiment_counts = (
    hops_queryset
    .values('experiment_id')  # Group by experiment_id
    .annotate(record_count=Count('id'))  # Count the number of records per group
    .order_by()  # Remove default ordering, if any
)

# Convert the result to a dictionary with experiment_id as key and record_count as value
experiment_counts_dict = {entry['experiment_id']: entry['record_count'] for entry in unique_experiment_counts}

KeyError: '3'